<a href="https://colab.research.google.com/github/Majidghne/Projet-RAG-Retrieval-Augmented-Generation-/blob/main/Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet RAG (Retrieval-Augmented Generation) avec Optimisation du Chunking**

Ce notebook présente une implémentation de Retrieval-Augmented Generation (RAG) en utilisant ChromaDB comme base de données vectorielle et l'API d'OpenRouter pour la génération de texte. L'objectif principal est de démontrer l'importance de l'optimisation du *chunking* (découpage du texte) pour améliorer la pertinence des réponses du modèle. Nous explorerons différentes stratégies de *chunking* et leurs impacts sur les résultats.

## **Étapes clés du projet :**

1.  **Installation des Dépendances :** Mise en place de toutes les bibliothèques nécessaires.
2.  **Lecture de Documents :** Fonctions pour lire différents formats de fichiers (PDF, DOCX, TXT).
3.  **Ingestion des Données :** Téléchargement et traitement d'un document PDF dans ChromaDB.
4.  **Implémentation RAG Initiale :** Démonstration d'un système RAG basique avec un *chunking* simple.
5.  **Optimisation du Chunking :** Exploration de deux méthodes avancées (*sentence-based* et *LangChain RecursiveCharacterTextSplitter*) pour un découpage plus sémantique.
6.  **Comparaison des Résultats :** Analyse de l'impact des différentes techniques de *chunking* sur la qualité des réponses.

## **1. Installation des Dépendances**

Nous commençons par installer les bibliothèques Python nécessaires pour le traitement des documents, la création d'embeddings, la base de données vectorielle (ChromaDB) et l'accès à l'API d'OpenAI/OpenRouter.

In [ ]:
# Installer les bibliothèques nécessaires
!pip install -qU chromadb # Base de données vectorielle pour le stockage des embeddings
!pip install -qU openai # Accès à l'API OpenAI/OpenRouter pour les modèles de langage
!pip install -qU pypdf2 # Pour la lecture et l'extraction de texte à partir de fichiers PDF
!pip install -qU python-docx # Pour la lecture et l'extraction de texte à partir de fichiers Word
!pip install -qU sentence-transformers # Pour la création d'embeddings (représentations vectorielles de texte)

## **2. Fonctions de Lecture de Documents**

Ces fonctions permettent de lire le contenu textuel de différents types de fichiers (PDF, Word, TXT). Une fonction unifiée `read_document` gère la détection du format et l'appel de la fonction de lecture appropriée.

In [ ]:
import PyPDF2 # Importe la bibliothèque pour la lecture de fichiers PDF
import docx # Importe la bibliothèque pour la lecture de fichiers DOCX
import os # Importe le module os pour les opérations sur les chemins de fichiers

def read_text_file(file_path: str):
    """Lire le contenu d'un fichier texte"""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def read_pdf_file(file_path: str):
  """Lire le contenu d'un fichier PDF"""
  text=""
  with open(file_path, 'rb') as file: # Ouvre le fichier PDF en mode binaire
    pdf_reader=PyPDF2.PdfReader(file) # Crée un objet PdfReader
    for page in pdf_reader.pages: # Itère sur chaque page du PDF
      text += page.extract_text() + "\n" # Extrait le texte de la page et l'ajoute
  return text

def read_docx_file(file_path: str):
  """Lire le contenu d'un fichier Word"""
  doc = docx.Document(file_path) # Ouvre le document Word
  return "\n".join([paragraph.text for paragraph in doc.paragraphs]) # Extrait le texte de chaque paragraphe et le joint

In [ ]:
def read_document(file_path: str):
    """Lit le contenu d'un document en fonction de son extension de fichier."""
    _, file_extension = os.path.splitext(file_path) # Extrait l'extension du fichier
    file_extension = file_extension.lower() # Convertit l'extension en minuscules
    if file_extension == ".txt":
        return read_text_file(file_path)
    elif file_extension == ".pdf":
        return read_pdf_file(file_path)
    elif file_extension == ".docx":
        return read_docx_file(file_path)
    else:
        raise ValueError(f"Format de fichier non supporté: {file_extension}")

## **3. Téléchargement et Traitement du Document**

Nous téléchargeons un document PDF à partir de Google Drive. Ce document servira de source de connaissances pour notre système RAG. Nous le lisons et affichons un extrait pour vérification.

In [ ]:
# Télécharge le fichier PDF depuis Google Drive
!gdown "https://drive.google.com/uc?id=1TywvYowEeL49qd-HKjr9-RUzjexz0N94"

Downloading...
From: https://drive.google.com/uc?id=1TywvYowEeL49qd-HKjr9-RUzjexz0N94
To: /content/thermodynamique-livre.pdf
100% 39.6M/39.6M [00:00<00:00, 92.7MB/s]


In [ ]:
# Lister les fichiers pdf dans /content
files = [f for f in os.listdir("/content") if f.endswith(".pdf")] # Filtre les fichiers pour ne garder que les PDF

In [ ]:
# Construit le chemin complet du dernier fichier PDF trouvé (hypothèse qu'il s'agit du bon fichier)
file_path = os.path.join("/content", files[-1])

print(file_path)

text = read_document(file_path) # Lit le contenu du document PDF

print("\n======Extracted PDF Content=======\n")
print(text[:500]) # Affiche les 500 premiers caractères du texte extrait pour vérification

/content/thermodynamique-livre-gratuit.pdf

======Extracted PDF Content=======

THERMODYNAMIQUE
DE L ’INGÉNIEUR
TROISIÈME ÉDITION
OLIVIER CLEYNEN
THERMODYNAMIQUE.FR
Ce livre est gratuit.
Vous pouvez le télécharger, le lire, le partager, et même le remixer : il est
publié sous licence CC by-sa . S’il vous est utile, pensez à le citer — c’est
important.
Si vous en avez les moyens, l’achat de la version
imprimée ou du pdfpayant aide à faire vivre le
projet. (Aujourd’hui, parmi les lecteurs et lectrices,
environ 1 sur 400 le fait — merci du fond du cœur
si vous en faites partie


## **4. Implémentation RAG Initiale : Chunking Basique**

Dans cette première approche, nous utilisons une méthode de *chunking* simple, basée sur une taille de morceau fixe avec un chevauchement. Nous définissons également les fonctions pour la recherche sémantique avec ChromaDB et la génération de réponses via l'API OpenRouter.

### **Fonction de Chunking Basique (`split_text`)**

Cette fonction découpe le texte en morceaux de taille fixe (500 caractères par défaut) avec un chevauchement (100 caractères par défaut). Cette méthode est simple mais peut souvent entraîner une perte de cohérence sémantique en coupant les phrases ou les idées au milieu.

In [ ]:
def split_text(text: str, chunk_size: int = 500, chunk_overlap: int = 100):
  """Sépare le texte en morceaux plus petits avec une taille fixe et un chevauchement."""
  text = text.replace("\n", " ").strip() # Remplace les sauts de ligne par des espaces et nettoie le texte
  chunks = []
  start = 0
  length = len(text)

  while start < length:
        end = min(start + chunk_size, length) # Détermine la fin du morceau, sans dépasser la longueur du texte
        chunk = text[start:end].strip() # Extrait le morceau
        if chunk:
            chunks.append(chunk) # Ajoute le morceau si non vide
        start += chunk_size - chunk_overlap # Avance le début du prochain morceau en considérant le chevauchement

  return chunks

In [ ]:
# Exemple de découpage avec la fonction split_text
sample = "This is a very long paragraph of text that you want to split into smaller chunks for embedding or storage."
chunks = split_text(sample, chunk_size=10, chunk_overlap=2) # Teste avec une petite taille de morceau
print(chunks)

['This is a', 'a very lon', 'ong paragr', 'graph of t', 'text that', 'at you wan', 'ant to spl', 'plit into', 'o smaller', 'r chunks f', 'for embed', 'edding or', 'r storage.', 'e.']


In [ ]:
# Applique le chunking basique au texte complet du document
chunks = split_text(text, chunk_size=500, chunk_overlap=50)

print("Chunk-01", chunks[0]) # Affiche le premier morceau
print("Chunk-02", chunks[1]) # Affiche le deuxième morceau
print("Number of Chunks", len(chunks)) # Affiche le nombre total de morceaux

Chunk-01 THERMODYNAMIQUE DE L ’INGÉNIEUR TROISIÈME ÉDITION OLIVIER CLEYNEN THERMODYNAMIQUE.FR Ce livre est gratuit. Vous pouvez le télécharger, le lire, le partager, et même le remixer : il est publié sous licence CC by-sa . S’il vous est utile, pensez à le citer — c’est important. Si vous en avez les moyens, l’achat de la version imprimée ou du pdfpayant aide à faire vivre le projet. (Aujourd’hui, parmi les lecteurs et lectrices, environ 1 sur 400 le fait — merci du fond du cœur si vous en faites partie
Chunk-02 t — merci du fond du cœur si vous en faites partie!) Bonne lecture! Olivier Cleynen, l’auteur. Thermodynamique de l’ingénieur Olivier Cleynen Thermodynamique de l’ingénieur Troisième édition, 2021 par Olivier Cleynen ISBN: 9781794848207 Copyright 2015, 2018, 2021, 2025 Olivier Cleynen (les éditions de 2015 et 2018 étaient publiées par Framasoft/Framabook) Thermodynamique de l’ingénieur est placé sous licence Creative Commons : c b a Attribution – Partage dans les Mêmes Conditi

### **Initialisation de ChromaDB**

Nous configurons notre base de données vectorielle persistante (ChromaDB) et définissons une fonction d'embedding (`all-MiniLM-L6-v2`) pour transformer nos morceaux de texte en vecteurs numériques.

In [ ]:
import chromadb # Importe la bibliothèque ChromaDB

from chromadb.utils import embedding_functions # Importe les fonctions d'embedding de ChromaDB

client = chromadb.PersistentClient(path="chroma_db") # Initialise un client ChromaDB persistant qui stocke les données localement dans le dossier "chroma_db"

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2" # Utilise le modèle "all-MiniLM-L6-v2" pour créer les embeddings
)

collection = client.get_or_create_collection(
    name="documents_collection", # Nom de la collection où les documents seront stockés
    embedding_function=sentence_transformer_ef # Fonction d'embedding à utiliser pour cette collection
)

### **Traitement du Document pour ChromaDB**

La fonction `process_document` combine la lecture, le *chunking* et la préparation des métadonnées (source, numéro de morceau) pour l'ingestion dans ChromaDB.

In [ ]:
def process_document(file_path: str):
  """Traite un document en le lisant, le découpant (chunking) et préparant les métadonnées pour ChromaDB."""
  try:
        content = read_document(file_path) # Lit le contenu du document

        chunks = split_text(content) # Découpe le contenu en morceaux en utilisant la fonction de chunking basique

        file_name = os.path.basename(file_path) # Extrait le nom du fichier
        metadatas = [{"source": file_name, "chunk": i} for i in range(len(chunks))] # Crée des métadonnées pour chaque morceau (source et numéro de morceau)
        ids = [f"{file_name}_chunk_{i}" for i in range(len(chunks))] # Génère des IDs uniques pour chaque morceau

        return ids, chunks, metadatas # Retourne les IDs, les morceaux et leurs métadonnées
  except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return [], [], []

### **Vérification des Chunks Initiaux**

In [ ]:
# Traite le document pour obtenir les IDs, les chunks et les métadonnées
ids, chunks, metadatas = process_document(file_path)

In [ ]:
# Affiche les informations du premier chunk pour vérification
print("id[0] -> ", ids[0])
print("metadatas[0] -> ", metadatas[0])
print("chunks[0] -> ", chunks[0])

id[0] ->  thermodynamique-livre-gratuit.pdf_chunk_0
metadatas[0] ->  {'source': 'thermodynamique-livre-gratuit.pdf', 'chunk': 0}
chunks[0] ->  THERMODYNAMIQUE DE L ’INGÉNIEUR TROISIÈME ÉDITION OLIVIER CLEYNEN THERMODYNAMIQUE.FR Ce livre est gratuit. Vous pouvez le télécharger, le lire, le partager, et même le remixer : il est publié sous licence CC by-sa . S’il vous est utile, pensez à le citer — c’est important. Si vous en avez les moyens, l’achat de la version imprimée ou du pdfpayant aide à faire vivre le projet. (Aujourd’hui, parmi les lecteurs et lectrices, environ 1 sur 400 le fait — merci du fond du cœur si vous en faites partie


In [ ]:
# Affiche le nombre total de chunks générés
len(chunks)

1756

### **Ajout des Chunks à ChromaDB**

Les morceaux de texte sont maintenant convertis en embeddings et stockés dans la collection ChromaDB, prêts pour la recherche sémantique.

In [ ]:
# Ajoute les documents (chunks), les métadonnées et les IDs à la collection ChromaDB
collection.add(documents=chunks, metadatas=metadatas, ids=ids)

### **Fonctions de Recherche Sémantique et de RAG**

Ces fonctions gèrent la recherche des morceaux les plus pertinents dans ChromaDB (`semantic_search`), la construction du contexte à partir de ces morceaux (`get_context_with_sources`), et l'appel au modèle de langage pour générer une réponse (`rag_answer`).

In [ ]:
def semantic_search(collection, query: str, n_results: int = 2):
    """Effectue une recherche sémantique dans la collection ChromaDB."""
    return collection.query(
        query_texts=[query], # La requête de recherche
        n_results=n_results, # Nombre de résultats les plus pertinents à retourner
        include=["documents", "metadatas"] # Inclut le contenu des documents et leurs métadonnées dans les résultats
    )

def get_context_with_sources(results):
    """Extrait le contexte et les sources des résultats de recherche."""
    if not results or not results.get("documents") or not results["documents"][0]:
        return "", []

    context = "\n\n".join(results["documents"][0]) # Concatène les morceaux de document trouvés pour former le contexte

    seen = set()
    sources = []
    for meta in results["metadatas"][0]:
        label = f"{meta.get('source','?')} (chunk {meta.get('chunk','?')})" # Formate l'étiquette de la source
        if label not in seen:
            seen.add(label)
            sources.append(label) # Ajoute la source si elle n'a pas déjà été vue

    return context, sources

def ask(collection, query: str, n_results: int = 2):
    """Fonction utilitaire pour effectuer une recherche, construire le contexte et afficher les sources."""

    results = semantic_search(collection, query, n_results=n_results)

    context, sources = get_context_with_sources(results)

    print("\n=== CONTEXT ===\n")
    print(context or "[No matching text found]")

    print("\n=== SOURCES ===")
    if sources:
        for i, s in enumerate(sources, 1):
            print(f"{i}. {s}")
    else:
        print("[No sources]")

    return context, sources

In [ ]:
# Exemple de question posée avec le chunking basique
query = "Quelle est un système fermé?"
context, sources = ask(collection, query, n_results=5) # Effectue la recherche et affiche le contexte et les sources


=== CONTEXT ===

es 𝑞kérosène= 300,2kgh−1(valeur réa- liste). Thermodynamique de l’ingénieur par Olivier Cleynen Chapitre 2 Les systèmes fermés ou Petit traité de comptabilité énergétique 32 Chapitre 2 Chapitre 2 – Les systèmes fermés 2.1 Pourquoi utiliser un système fermé? 33 2.2 Conventions de comptabilité 34 2.2.1 Le système fermé 34 2.2.2 Conventions de signe 35 2.3 Le premier principe dans un système fermé 35 2.4 Quanti fier le travail avec un système fermé 36 2.4.1 Le travail en fonction du volume, avec un

systèmes fermés )se propose de répondre à deux questions : •Comment quanti fier le travail que peut recevoir et fournir un corps de massefixe? •Qu’est-ce que la réversibilité, pourquoi et comment l’atteindre? 2.1 Pourquoi utiliser un système fermé? À partir de maintenant, nous voulons décrire et quanti fier les transferts d’énergie dans les fluides. Nous pouvons adopter deux points de vue différents pour observer le fluide : •Soit nous « découpons » un petit morceau de masse,

### **Configuration de l'API OpenRouter**

Nous utilisons OpenRouter pour accéder à des modèles de langage, en configurant la clé API et le nom du modèle.

In [ ]:
from dotenv import load_dotenv # Importe load_dotenv pour charger les variables d'environnement

# Crée ou écrase un fichier .env avec une clé API placeholder
with open(".env", "w") as f:
    f.write('OPEN_ROUTER_API_KEY="VOTRE_CLE_ICI"') # L'utilisateur doit remplacer "VOTRE_CLE_ICI" par sa vraie clé

load_dotenv() # Charge les variables d'environnement depuis le fichier .env

True

In [ ]:
OPEN_ROUTER_API_KEY = os.getenv("OPEN_ROUTER_API_KEY") # Récupère la clé API depuis les variables d'environnement
OPEN_ROUTER_MODEL_NAME = "openai/gpt-oss-120b:free" # Définit le nom du modèle OpenRouter à utiliser

In [ ]:
from openai import OpenAI # Importe la classe OpenAI

client = OpenAI(
  base_url = "https://openrouter.ai/api/v1", # Définit l'URL de base pour l'API OpenRouter
  api_key = OPEN_ROUTER_API_KEY, # Utilise la clé API configurée
)

In [ ]:
SYSTEM_PROMPT = (
    "You are a helpful assistant for retrieval-augmented generation (RAG).\n" # Rôle de l'assistant
    "Answer ONLY using the provided context. " # Instruction cruciale: ne répondre qu'avec le contexte fourni
    "If the answer is not found in the context, say: "
    "'I don't know based on the provided documents.'" # Réponse si l'information n'est pas dans le contexte
)

def build_messages(context: str, question: str):
    """Construit la liste des messages pour l'API OpenAI/OpenRouter."""
    return [
        {"role": "system", "content": SYSTEM_PROMPT}, # Message système pour définir le comportement de l'IA
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"} # Message utilisateur incluant le contexte et la question
    ]

In [ ]:
def rag_answer(collection, query: str, n_results: int = 4, model: str = OPEN_ROUTER_MODEL_NAME):
    """Exécute la recherche sémantique, génère une réponse avec l'IA et affiche les résultats."""
    results = semantic_search(collection, query, n_results) # Effectue la recherche sémantique
    context, sources = get_context_with_sources(results) # Extrait le contexte et les sources

    if not context.strip(): # Si aucun contexte pertinent n'est trouvé
        print("No relevant context found.")
        return "", []

    messages = build_messages(context, query) # Construit les messages pour l'API de l'IA

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2, # Température basse pour des réponses plus déterministes
        max_tokens=512 # Limite la longueur de la réponse
    )

    answer = response.choices[0].message.content.strip() # Extrait le contenu de la réponse de l'IA

    print("\n=== ANSWER ===\n")
    print(answer or "[No answer generated]")

    print("\n=== SOURCES ===")
    if sources:
        for i, s in enumerate(sources, 1):
            print(f"{i}. {s}")
    else:
        print("[No sources found]")

    return answer, sources

### **Exemple de Réponse RAG (Chunking Basique)**

Nous posons une question et observons la réponse générée avec le *chunking* basique. Notez les sources utilisées.

In [ ]:
# Exemple de question pour tester le RAG avec le chunking basique
query = "Cycles moteur à vapeur"
rag_answer(collection, query, n_results=2)


=== ANSWER ===

Les cycles moteur à vapeur sont étudiés dans le chapitre 9 du cours « Thermodynamique de l’ingénieur » afin de répondre à deux grandes questions :

1. **Pourquoi et comment utilise‑t‑on les moteurs à vapeur aujourd’hui ?**  
   - Le texte rappelle que, depuis longtemps, on a tenté d’exploiter la chaleur sur différents fluides (par exemple l’air atmosphérique) pour produire de la puissance motrice.  
   - Comparé à l’air, la vapeur d’eau présente des **avantages** (notamment en termes de capacité à absorber et à libérer de la chaleur) qui justifient son emploi dans les cycles thermodynamiques industriels.

2. **Pourquoi s’éloigne‑t‑on des cycles idéaux et comment quantifie‑t‑on ces compromis ?**  
   - Le chapitre vise à expliquer les raisons pour lesquelles les cycles réels diffèrent des cycles idéaux (pertes, contraintes de conception, etc.) et à montrer comment on mesure l’impact de ces écarts sur la performance du moteur à vapeur.

En résumé, les cycles moteur à vap

('Les cycles moteur à vapeur sont étudiés dans le chapitre\u202f9 du cours «\u202fThermodynamique de l’ingénieur\u202f» afin de répondre à deux grandes questions\u202f:\n\n1. **Pourquoi et comment utilise‑t‑on les moteurs à vapeur aujourd’hui\u202f?**  \n   - Le texte rappelle que, depuis longtemps, on a tenté d’exploiter la chaleur sur différents fluides (par exemple l’air atmosphérique) pour produire de la puissance motrice.  \n   - Comparé à l’air, la vapeur d’eau présente des **avantages** (notamment en termes de capacité à absorber et à libérer de la chaleur) qui justifient son emploi dans les cycles thermodynamiques industriels.\n\n2. **Pourquoi s’éloigne‑t‑on des cycles idéaux et comment quantifie‑t‑on ces compromis\u202f?**  \n   - Le chapitre vise à expliquer les raisons pour lesquelles les cycles réels diffèrent des cycles idéaux (pertes, contraintes de conception, etc.) et à montrer comment on mesure l’impact de ces écarts sur la performance du moteur à vapeur.\n\nEn résumé,

In [ ]:
# Autre exemple de question avec le chunking basique
query = "cycle de Diesel?"
rag_answer(collection, query, n_results=3)


=== ANSWER ===

Le cycle de Diesel est le cycle thermodynamique utilisé par le moteur Diesel.  
Selon le texte fourni, il se caractérise par :

* **Un fort taux de compression** : la compression de l’air est très élevée afin d’augmenter sa température avant la combustion.  
* **Une combustion indépendante de l’admission d’air** : la combustion intervient après la compression, à pression (ou température) quasi‑constante, ce qui permet une qualité de combustion supérieure à celle du cycle d’Otto.  
* **Un rendement légèrement inférieur à celui du cycle d’Otto** : bien que le taux de compression soit plus élevé, le cycle de Diesel présente un rendement plus faible que le cycle d’Otto en raison de la température maximale plus élevée atteinte pendant le processus.

En résumé, le cycle de Diesel repose sur un taux de compression élevé et une combustion à pression (ou température) quasi‑constante, offrant une meilleure qualité de combustion mais un rendement global inférieur à celui du cycle

('Le cycle de Diesel est le cycle thermodynamique utilisé par le moteur Diesel.  \nSelon le texte fourni, il se caractérise par\u202f:\n\n* **Un fort taux de compression**\u202f: la compression de l’air est très élevée afin d’augmenter sa température avant la combustion.  \n* **Une combustion indépendante de l’admission d’air**\u202f: la combustion intervient après la compression, à pression (ou température) quasi‑constante, ce qui permet une qualité de combustion supérieure à celle du cycle d’Otto.  \n* **Un rendement légèrement inférieur à celui du cycle d’Otto**\u202f: bien que le taux de compression soit plus élevé, le cycle de Diesel présente un rendement plus faible que le cycle d’Otto en raison de la température maximale plus élevée atteinte pendant le processus.\n\nEn résumé, le cycle de Diesel repose sur un taux de compression élevé et une combustion à pression (ou température) quasi‑constante, offrant une meilleure qualité de combustion mais un rendement global inférieur à ce

In [ ]:
# Troisième exemple de question avec le chunking basique
query = "Quelle est une évolution rapide?"
rag_answer(collection, query, n_results=5)


=== ANSWER ===

Une évolution rapide désigne un aller‑retour effectué très vite, de façon non quasi‑statique. Dans ce type de processus la pression finale à chaque trajet est plus élevée que celle qui aurait été obtenue lors d’une évolution lente (quasi‑équilibrée). En d’autres termes, la transformation se fait rapidement, de façon irréversible, et la pression ne suit pas les mêmes valeurs que lors d’une évolution lente.

=== SOURCES ===
1. thermodynamique-livre-gratuit.pdf (chunk 233)
2. thermodynamique-livre-gratuit.pdf (chunk 1065)
3. thermodynamique-livre-gratuit.pdf (chunk 218)
4. thermodynamique-livre-gratuit.pdf (chunk 505)
5. thermodynamique-livre-gratuit.pdf (chunk 1154)


('Une évolution rapide désigne un aller‑retour effectué très vite, de façon non quasi‑statique. Dans ce type de processus la pression finale à chaque trajet est plus élevée que celle qui aurait été obtenue lors d’une évolution lente (quasi‑équilibrée). En d’autres termes, la transformation se fait rapidement, de façon irréversible, et la pression ne suit pas les mêmes valeurs que lors d’une évolution lente.',
 ['thermodynamique-livre-gratuit.pdf (chunk 233)',
  'thermodynamique-livre-gratuit.pdf (chunk 1065)',
  'thermodynamique-livre-gratuit.pdf (chunk 218)',
  'thermodynamique-livre-gratuit.pdf (chunk 505)',
  'thermodynamique-livre-gratuit.pdf (chunk 1154)'])

## **5. Optimisation du Chunking**

Le *chunking* basique peut entraîner des problèmes de cohérence sémantique :
* Coupe au milieu d’une phrase
* Peut séparer une définition en deux parties
* Peut séparer une question et sa réponse
* Entraîne une perte de cohérence sémantique

 Par conséquent, les embeddings sont moins pertinents et la recherche devient moins précise.

Alors, nous allons explorer deux solutions pour améliorer la qualité du découpage des textes.

### **Solution 1 : Chunking basé sur les phrases (NLTK)**

Cette méthode vise à conserver l'intégrité des phrases. Nous utilisons la bibliothèque `NLTK` pour la tokenisation des phrases, puis nous construisons des morceaux en regroupant des phrases jusqu'à atteindre une taille maximale. Cela réduit le risque de couper des informations importantes au milieu d'une phrase.

### **Installation de NLTK**

In [ ]:
# Installe la bibliothèque NLTK
!pip install nltk
import nltk
nltk.download('punkt') # Télécharge le tokenizer 'punkt' pour le découpage de phrases
nltk.download('punkt_tab') # Télécharge une version tabulaire du tokenizer (si nécessaire)
from nltk.tokenize import sent_tokenize # Importe la fonction de tokenization de phrases

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### **Fonction de Chunking par Phrases (`split_text_by_sentences`)**

In [ ]:
def split_text_by_sentences(text, max_chunk_size=500):
    """Découpe le texte en morceaux en respectant les limites de phrases."""
    sentences = sent_tokenize(text) # Découpe le texte en phrases

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # Si ajouter la phrase actuelle ne dépasse pas la taille maximale du morceau
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += " " + sentence # Ajoute la phrase au morceau courant
        else:
            chunks.append(current_chunk.strip()) # Ajoute le morceau courant aux chunks
            current_chunk = sentence # Commence un nouveau morceau avec la phrase actuelle

    if current_chunk: # Ajoute le dernier morceau s'il n'est pas vide
        chunks.append(current_chunk.strip())

    return chunks

### **Application et Vérification des Nouveaux Chunks**

Nous appliquons cette nouvelle méthode de *chunking* au texte du document.

In [ ]:
# Applique le découpage par phrases au texte du document
chunks = split_text_by_sentences(text)
print("Nombre de chunks :", len(chunks)) # Affiche le nombre de chunks après découpage par phrases

Nombre de chunks : 1592


### **Réinitialisation et Rechargement de la Collection ChromaDB**

Pour comparer les résultats, nous devons d'abord vider l'ancienne collection ChromaDB et la re-remplir avec les nouveaux chunks obtenus par la méthode basée sur les phrases. Nous allons d'abord supprimer la collection existante si elle existe, puis la recréer. **Note:** Dans cet exemple, les étapes `ids, chunks, metadatas = process_document(file_path)` et `collection.add(...)` doivent être relancées avec la nouvelle fonction de chunking pour refléter les changements.

In [ ]:
def process_document2(file_path: str):
  """Traite un document en utilisant le chunking basé sur les phrases pour ChromaDB."""
  try:
        content = read_document(file_path) # Lit le contenu du document

        chunks = split_text_by_sentences(content) # Utilise la nouvelle fonction de chunking par phrases

        file_name = os.path.basename(file_path) # Extrait le nom du fichier
        metadatas = [{"source": file_name, "chunk": i} for i in range(len(chunks))] # Crée les métadonnées
        ids = [f"{file_name}_chunk_{i}" for i in range(len(chunks))] # Génère les IDs

        return ids, chunks, metadatas
  except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return [], [], []


In [ ]:
# Traite le document avec le chunking basé sur les phrases
ids, chunks, metadatas = process_document2(file_path)

In [ ]:
# Ajoute les nouveaux chunks (basés sur les phrases) à la collection ChromaDB. Note: cela va s'ajouter aux chunks existants si la collection n'a pas été vidée.
collection.add(documents=chunks, metadatas=metadatas, ids=ids)

### **Exemple de Réponse RAG (Chunking NLTK)**

Nous posons la même question et observons la réponse générée avec le *chunking* basé sur les phrases. Comparons la qualité des réponses et des sources avec la méthode précédente.

In [ ]:
# Pose la même question avec les chunks basés sur les phrases
query = "Cycles moteur à vapeur"
rag_answer(collection, query, n_results=2)


=== ANSWER ===

Les cycles moteur à vapeur sont étudiés dans le chapitre 9 afin de répondre à deux questions principales :  

* **Pourquoi et comment utilise‑t‑on des moteurs à vapeur aujourd’hui ?**  
  On a, à plusieurs reprises, tenté de faire agir la chaleur sur l’air atmosphérique pour produire de la puissance motrice. Le texte indique que ce gaz (l’air) présente, par rapport à la vapeur d’eau, à la fois des avantages et des inconvénients. L’un des avantages notables de l’air, comparé à la vapeur, est mentionné dès le premier point (« Il présente, relativement à la vapeur d’eau, un avantage notable… »).  

* **Pour quelles raisons s’éloigne‑t‑on des cycles idéaux et comment quantifie‑t‑on ces compromis ?**  
  Le chapitre vise à expliquer les écarts entre les cycles idéaux et les cycles réels, ainsi que les méthodes de quantification de ces compromis.

En résumé, le « cycles moteur à vapeur » désignent l’ensemble des procédés thermodynamiques étudiés pour comprendre l’utilisation

('Les cycles moteur à vapeur sont étudiés dans le chapitre\u202f9 afin de répondre à deux questions principales\u202f:  \n\n* **Pourquoi et comment utilise‑t‑on des moteurs à vapeur aujourd’hui\u202f?**  \n  On a, à plusieurs reprises, tenté de faire agir la chaleur sur l’air atmosphérique pour produire de la puissance motrice. Le texte indique que ce gaz (l’air) présente, par rapport à la vapeur d’eau, à la fois des avantages et des inconvénients. L’un des avantages notables de l’air, comparé à la vapeur, est mentionné dès le premier point («\u202fIl présente, relativement à la vapeur d’eau, un avantage notable…\u202f»).  \n\n* **Pour quelles raisons s’éloigne‑t‑on des cycles idéaux et comment quantifie‑t‑on ces compromis\u202f?**  \n  Le chapitre vise à expliquer les écarts entre les cycles idéaux et les cycles réels, ainsi que les méthodes de quantification de ces compromis.\n\nEn résumé, le «\u202fcycles moteur à vapeur\u202f» désignent l’ensemble des procédés thermodynamiques étud

In [ ]:
# Autre question avec les chunks basés sur les phrases
query = "cycle de Diesel?"
rag_answer(collection, query, n_results=3)


=== ANSWER ===

Le cycle de Diesel est le cycle thermodynamique utilisé par le moteur Diesel.  
Il se caractérise :

* **Un fort taux de compression** : le moteur comprime l’air avant l’injection du carburant, ce qui élève fortement la température de l’air.  
* **Une combustion à température élevée** : le carburant est injecté dans l’air déjà chaud et la combustion se produit à pression (ou à température) quasi‑constante.  

Grâce à ce taux de compression très élevé, le cycle de Diesel offre une **qualité de combustion supérieure** à celle du cycle d’Otto. Cependant, comme la température maximale atteinte est plus élevée, le rendement du cycle de Diesel est **légèrement inférieur** à celui du cycle d’Otto.  

En résumé, le cycle de Diesel repose sur une compression importante de l’air suivie d’une injection de carburant et d’une combustion à pression quasi‑constante, ce qui le rend plus efficace en termes de combustion mais légèrement moins performant en rendement thermique que le cyc

('Le cycle de Diesel est le cycle thermodynamique utilisé par le moteur Diesel.  \nIl se caractérise\u202f:\n\n* **Un fort taux de compression**\u202f: le moteur comprime l’air avant l’injection du carburant, ce qui élève fortement la température de l’air.  \n* **Une combustion à température élevée**\u202f: le carburant est injecté dans l’air déjà chaud et la combustion se produit à pression (ou à température) quasi‑constante.  \n\nGrâce à ce taux de compression très élevé, le cycle de Diesel offre une **qualité de combustion supérieure** à celle du cycle d’Otto. Cependant, comme la température maximale atteinte est plus élevée, le rendement du cycle de Diesel est **légèrement inférieur** à celui du cycle d’Otto.  \n\nEn résumé, le cycle de Diesel repose sur une compression importante de l’air suivie d’une injection de carburant et d’une combustion à pression quasi‑constante, ce qui le rend plus efficace en termes de combustion mais légèrement moins performant en rendement thermique qu

In [ ]:
# Troisième question avec les chunks basés sur les phrases
query = "Quelle est une évolution rapide?"
rag_answer(collection, query, n_results=5)


=== ANSWER ===

Une évolution rapide est un processus thermodynamique effectué très rapidement, de façon brusque. Contrairement aux évolutions lentes où la pression suit toujours les mêmes valeurs lors des allers‑retours, lors d’une évolution rapide la pression finale à chaque trajet est plus élevée que celle qui aurait été obtenue avec un trajet lent. En d’autres termes, le changement de l’état du système se fait en un temps très court, entraînant des écarts de pression (et d’autres grandeurs) par rapport aux valeurs quasi‑équilibrées observées lors d’une évolution lente.

=== SOURCES ===
1. thermodynamique-livre-gratuit.pdf (chunk 233)
2. thermodynamique-livre-gratuit.pdf (chunk 1065)
3. thermodynamique-livre-gratuit.pdf (chunk 218)
4. thermodynamique-livre-gratuit.pdf (chunk 505)
5. thermodynamique-livre-gratuit.pdf (chunk 1154)


('Une évolution rapide est un processus thermodynamique effectué très rapidement, de façon brusque. Contrairement aux évolutions lentes où la pression suit toujours les mêmes valeurs lors des allers‑retours, lors d’une évolution rapide la pression finale à chaque trajet est plus élevée que celle qui aurait été obtenue avec un trajet lent. En d’autres termes, le changement de l’état du système se fait en un temps très court, entraînant des écarts de pression (et d’autres grandeurs) par rapport aux valeurs quasi‑équilibrées observées lors d’une évolution lente.',
 ['thermodynamique-livre-gratuit.pdf (chunk 233)',
  'thermodynamique-livre-gratuit.pdf (chunk 1065)',
  'thermodynamique-livre-gratuit.pdf (chunk 218)',
  'thermodynamique-livre-gratuit.pdf (chunk 505)',
  'thermodynamique-livre-gratuit.pdf (chunk 1154)'])

### **Solution 2 : Chunking avancé avec LangChain `RecursiveCharacterTextSplitter`**

LangChain est une bibliothèque Python conçue pour construire des applications avec des LLM :

* RAG (Retrieval-Augmented Generation)
* Agents
* Chatbots avec mémoire
* Pipelines LLM complexes

Dans notre cas, on l’utiliserait uniquement pour faire un meilleur découpage du texte (text splitting).

**`RecursiveCharacterTextSplitter`**

 Cet outil tente de découper le texte de manière hiérarchique en utilisant une liste de séparateurs, ce qui permet de préserver au maximum la structure sémantique du document.

### **Installation des Dépendances LangChain**

In [ ]:
# Installe les bibliothèques LangChain nécessaires
!pip install -q langchain langchain-text-splitters

### **Application et Vérification des Chunks avec LangChain**

Nous appliquons `RecursiveCharacterTextSplitter` au texte de notre document.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # Importe le découpeur de texte récursif de LangChain

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, # Taille maximale souhaitée pour chaque morceau
    chunk_overlap=50, # Chevauchement entre les morceaux pour préserver le contexte
    separators=["\n\n", "\n", ".", " ", ""] # Ordre des séparateurs à essayer pour le découpage (paragraphe, ligne, phrase, mot, caractère)
)

chunks = text_splitter.split_text(text) # Applique le découpeur au texte complet

print("Nombre de chunks :", len(chunks))
print("Chunk 1:\n", chunks[0])

Nombre de chunks : 1544
Chunk 1:
 THERMODYNAMIQUE
DE L ’INGÉNIEUR
TROISIÈME ÉDITION
OLIVIER CLEYNEN
THERMODYNAMIQUE.FR
Ce livre est gratuit.
Vous pouvez le télécharger, le lire, le partager, et même le remixer : il est
publié sous licence CC by-sa . S’il vous est utile, pensez à le citer — c’est
important.
Si vous en avez les moyens, l’achat de la version
imprimée ou du pdfpayant aide à faire vivre le
projet. (Aujourd’hui, parmi les lecteurs et lectrices,
environ 1 sur 400 le fait — merci du fond du cœur


### **Réinitialisation et Rechargement de la Collection ChromaDB (avec LangChain Chunks)**

Nous allons maintenant vider la collection ChromaDB et la re-remplir avec les chunks générés par LangChain.

In [ ]:
def process_document3(file_path: str):
  """Traite un document en utilisant le `RecursiveCharacterTextSplitter` de LangChain pour ChromaDB."""
  try:
        content = read_document(file_path) # Lit le contenu du document

        chunks = text_splitter.split_text(content) # Utilise la fonction de chunking de LangChain

        file_name = os.path.basename(file_path) # Extrait le nom du fichier
        metadatas = [{"source": file_name, "chunk": i} for i in range(len(chunks))] # Crée les métadonnées
        ids = [f"{file_name}_chunk_{i}" for i in range(len(chunks))] # Génère les IDs

        return ids, chunks, metadatas
  except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return [], [], []


In [ ]:
# Traite le document avec le chunking LangChain
ids, chunks, metadatas = process_document3(file_path)

In [ ]:
# Ajoute les nouveaux chunks (basés sur LangChain) à la collection ChromaDB
collection.add(documents=chunks, metadatas=metadatas, ids=ids)

### **Exemple de Réponse RAG (Chunking LangChain)**


In [ ]:
# Pose la question avec les chunks générés par LangChain
query = "Cycles moteur à vapeur"
rag_answer(collection, query, n_results=2)


=== ANSWER ===

Le chapitre 9 du cours « Thermodynamique de l’ingénieur » porte sur les **cycles moteur à vapeur**.  
Il a pour but de :

* expliquer **pourquoi et comment les moteurs à vapeur sont encore utilisés aujourd’hui** ;
* montrer **pour quelles raisons on s’éloigne des cycles idéaux** (contraintes réelles, pertes, rendements…) et **comment on quantifie ces compromis**.

Le texte introduit également une comparaison entre l’utilisation de la chaleur sur l’air atmosphérique et sur la vapeur d’eau, en soulignant que l’air présente, par rapport à la vapeur, des **avantages et des inconvénients** qui seront étudiés plus loin.

=== SOURCES ===
1. thermodynamique-livre-gratuit.pdf (chunk 1205)
2. thermodynamique-livre-gratuit.pdf (chunk 1206)


('Le chapitre\u202f9 du cours «\u202fThermodynamique de l’ingénieur\u202f» porte sur les **cycles moteur à vapeur**.  \nIl a pour but de :\n\n* expliquer **pourquoi et comment les moteurs à vapeur sont encore utilisés aujourd’hui** ;\n* montrer **pour quelles raisons on s’éloigne des cycles idéaux** (contraintes réelles, pertes, rendements…) et **comment on quantifie ces compromis**.\n\nLe texte introduit également une comparaison entre l’utilisation de la chaleur sur l’air atmosphérique et sur la vapeur d’eau, en soulignant que l’air présente, par rapport à la vapeur, des **avantages et des inconvénients** qui seront étudiés plus loin.',
 ['thermodynamique-livre-gratuit.pdf (chunk 1205)',
  'thermodynamique-livre-gratuit.pdf (chunk 1206)'])

In [ ]:
# Autre question avec les chunks générés par LangChain
query = "cycle de Diesel?"
rag_answer(collection, query, n_results=3)


=== ANSWER ===

Le cycle de Diesel est le cycle thermodynamique utilisé par le moteur Diesel.  
Il se caractérise :

* **Un fort taux de compression** : l’air est fortement comprimé avant l’injection du carburant, ce qui élève fortement sa température.  
* **Une combustion à pression (ou à température) constante** : le carburant est injecté dans l’air chaud et s’enflamme spontanément, la combustion se déroulant à pression (ou température) pratiquement constante.  
* **Une admission d’air uniquement** : le moteur ne reçoit que de l’air, le carburant étant ajouté après la compression.  

Ces propriétés donnent au cycle de Diesel un taux de compression et une qualité de combustion supérieurs à ceux du cycle d’Otto, même si son rendement thermique est légèrement inférieur lorsqu’on compare les températures maximales atteintes.  

En résumé, le cycle de Diesel repose sur une compression élevée de l’air suivie d’une injection et d’une combustion à pression constante, ce qui le différencie d

('Le cycle de Diesel est le cycle thermodynamique utilisé par le moteur Diesel.  \nIl se caractérise\u202f:\n\n* **Un fort taux de compression**\u202f: l’air est fortement comprimé avant l’injection du carburant, ce qui élève fortement sa température.  \n* **Une combustion à pression (ou à température) constante**\u202f: le carburant est injecté dans l’air chaud et s’enflamme spontanément, la combustion se déroulant à pression (ou température) pratiquement constante.  \n* **Une admission d’air uniquement**\u202f: le moteur ne reçoit que de l’air, le carburant étant ajouté après la compression.  \n\nCes propriétés donnent au cycle de Diesel un taux de compression et une qualité de combustion supérieurs à ceux du cycle d’Otto, même si son rendement thermique est légèrement inférieur lorsqu’on compare les températures maximales atteintes.  \n\nEn résumé, le cycle de Diesel repose sur une compression élevée de l’air suivie d’une injection et d’une combustion à pression constante, ce qui le

In [ ]:
# Troisième question avec les chunks générés par LangChain
query = "Quelle est une évolution rapide?"
rag_answer(collection, query, n_results=5)


=== ANSWER ===

Une « évolution rapide » désigne un processus thermodynamique effectué très rapidement, de façon brusque. Contrairement aux allers‑retours lents où la pression suit toujours les mêmes valeurs, lors d’une évolution rapide la pression finale à chaque trajet est plus élevée que celle qui aurait été obtenue dans un trajet lent. En d’autres termes, le changement se fait de façon très rapide, entraînant des écarts de pression et un caractère irréversible du processus.

=== SOURCES ===
1. thermodynamique-livre-gratuit.pdf (chunk 233)
2. thermodynamique-livre-gratuit.pdf (chunk 1065)
3. thermodynamique-livre-gratuit.pdf (chunk 218)
4. thermodynamique-livre-gratuit.pdf (chunk 505)
5. thermodynamique-livre-gratuit.pdf (chunk 1154)


('Une «\u202févolution rapide\u202f» désigne un processus thermodynamique effectué très rapidement, de façon brusque. Contrairement aux allers‑retours lents où la pression suit toujours les mêmes valeurs, lors d’une évolution rapide la pression finale à chaque trajet est plus élevée que celle qui aurait été obtenue dans un trajet lent. En d’autres termes, le changement se fait de façon très rapide, entraînant des écarts de pression et un caractère irréversible du processus.',
 ['thermodynamique-livre-gratuit.pdf (chunk 233)',
  'thermodynamique-livre-gratuit.pdf (chunk 1065)',
  'thermodynamique-livre-gratuit.pdf (chunk 218)',
  'thermodynamique-livre-gratuit.pdf (chunk 505)',
  'thermodynamique-livre-gratuit.pdf (chunk 1154)'])

## **Conclusion : L'Importance du Chunking**

À travers ces exemples, nous avons pu observer comment différentes stratégies de *chunking* peuvent influencer la qualité des réponses d'un système RAG. Un découpage intelligent du texte, qui respecte la cohérence sémantique, est crucial pour obtenir des embeddings plus pertinents et, par conséquent, des recherches plus précises et des réponses plus fidèles aux documents sources.

Le `RecursiveCharacterTextSplitter` de LangChain, avec sa stratégie hiérarchique de découpage, offre souvent les meilleurs résultats en équilibrant la taille des *chunks* et la préservation du contexte sémantique. Cela démontre que l'ingénierie du *chunking* est une étape fondamentale pour construire des applications RAG performantes et fiables.